In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import palettable

In [4]:
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt

def func2(x,a,b):
    return a+b/x

prefix = './Si'
## Readin DFT/PBE results
eig_ks = []
with open(prefix+'/pw/eigenval.xml','r') as f:
    line = f.readline()
    while line != '':
        eig_ks.append(float(line)*27.2114)
        line = f.readline()
homo = eig_ks[127]
eig_ks = eig_ks[32:184]

# Readin GW/PBE results
gw = []
arr = np.array([500,1000,1500,2000])
for i in arr:
    tmp = []
    with open(prefix+'/wfreq/std-'+str(i)+'-152st/std-'+str(i)+'.output', 'r') as f:
        lines = f.readlines()
        for j in range(len(lines)):
            if 'CONVERGENCE' in lines[j]:
                for k in range(j-29,j-3):
                    tmp = tmp + [float(x) for x in lines[k].split()]
    gw.append(tmp)
gw_fit = []
for st in range(len(gw[0])):
    propt, propv = curve_fit(func2,arr, [gw[i][st] for i in range(len(arr))])
    gw_fit.append(propt[0])
dif = [gw_fit[i]-eig_ks[i] for i in range(len(eig_ks))]

gw1000 = []
with open(prefix+'/wfreq/std-1000-152st/std-1000.output','r') as f:
    lines = f.readlines()
    for j in range(len(lines)):
        if 'CONVERGENCE' in lines[j]:
            for k in range(j-29,j-3):
                gw1000 = gw1000 + [float(x) for x in lines[k].split()]
gw2000 = []
with open(prefix+'/wfreq/std-2000-152st/std-2000.output','r') as f:
    lines = f.readlines()
    for j in range(len(lines)):
        if 'CONVERGENCE' in lines[j]:
            for k in range(j-29,j-3):
                gw2000 = gw2000 + [float(x) for x in lines[k].split()]
dif1000 = [gw1000[i]-eig_ks[i] for i in range(len(eig_ks))]
dif2000 = [gw2000[i]-eig_ks[i] for i in range(len(eig_ks))]

# kin-400-*.output
kin400 = []
arr4 = np.array([100,200,300,400])
for i in arr4:
    tmp = []
    with open(prefix+'/wfreq/kin-400-152st/kin-400-'+str(i)+'/kin-400-'+str(i)+'.output', 'r') as f:
        lines = f.readlines()
        for j in range(len(lines)):
            if 'CONVERGENCE' in lines[j]:
                for k in range(j-29,j-3):
                    tmp = tmp + [float(x) for x in lines[k].split()]
    kin400.append(tmp)
kin400_fit = []
for st in range(len(kin400[0])):
    propt, propv = curve_fit(func2, arr4+400, [kin400[i][st] for i in range(len(arr4))])
    kin400_fit.append(propt[0])
dif4 = [kin400_fit[i]-eig_ks[i] for i in range(len(eig_ks))]


for i in range(len(kin400_fit)):
    if 95 <= i <= 96:
        print("level %3d, gw1000:%8.4f, gw2000:%8.4f, gw_fit:%8.4f kin400_fit:%8.4f"
              %(i+33,gw1000[i],gw2000[i],gw_fit[i],kin400_fit[i]))
   

level 128, gw1000:  5.6971, gw2000:  5.5544, gw_fit:  5.4471 kin400_fit:  5.5340
level 129, gw1000:  7.0308, gw2000:  6.9063, gw_fit:  6.7878 kin400_fit:  6.8240
